 YOLO Model Eğitim ve Değerlendirme Akışı

Bu notebook, özel bir veri kümesi üzerinde YOLO nesne algılama modeli eğitme, doğrulama ve sonuçları paketleme adımlarını içerir.

**Adımlar:**
1.  **GPU Kontrolü:** Çalışma ortamında GPU olup olmadığını kontrol eder.
2.  **Veri Kümesi İndirme:** Roboflow'dan veri kümesini indirir.
3.  **Veri Kümesi Çıkarma:** İndirilen zip dosyasını çıkarır.
4.  **Veri Kümesi Yeniden Dengeleme ve Organizasyon:** Mevcut veri kümesini alır, resim ve etiketleri karıştırır ve belirtilen oranlara göre `train`, `valid`, `test` olarak yeniden böler. Yeni bir `data.yaml` dosyası oluşturur.
5.  **Ultralytics Kurulumu:** YOLO11 ve ilgili araçları içeren `ultralytics` kütüphanesini kurar.
6.  **YOLO Model Eğitimi:** Yeniden dengelenmiş veri kümesi ve belirtilen yapılandırma ile YOLO modelini eğitir. Eğitim sonrası en iyi model ile doğrulama yapar.
7.  **Sonuçları Arşivleme:** Eğitim çıktılarını (en iyi model ağırlıkları, loglar, grafikler vb.) toplar ve bir zip dosyası olarak paketler.

Bu hücre, Colab ortamında kullanılabilir bir NVIDIA GPU olup olmadığını kontrol etmek için nvidia-smi komutunu çalıştırır. Eğer bir GPU varsa, modeli, sürücü versiyonu ve bellek kullanımı gibi bilgileri listeler. Bu, model eğitimi gibi yoğun hesaplama gerektiren işlemler için önemlidir.

In [ ]:
# 1. GPU Kontrolü
# Bu komut, NVIDIA GPU'larının durumunu ve kullanılabilirliğini listeler.
# Eğer bir GPU varsa, modeli ve sürücü versiyonunu, bellek kullanımını vb. gösterir.
!nvidia-smi

Bu hücre, Roboflow platformundan sağlanan bir URL aracılığıyla özel bir veri kümesini indirir.
-O /content/data.zip: İndirilen dosyayı /content/ dizinine data.zip adıyla kaydeder.
https://app.roboflow.com/ds/uOodPMnwHn?key=8u2eVuAw14: Veri kümesinin indirme bağlantısı. key parametresi, bu özel veri kümesine erişim için bir API anahtarı içerir.

In [ ]:
# 2. Veri Kümesi İndirme
# Roboflow platformundan sağlanan URL aracılığıyla veri kümesini indirir.
# '-O /content/data.zip' seçeneği, indirilen dosyayı '/content/data.zip' olarak kaydeder.
# 'key' parametresi, özel veri kümesine erişim için bir API anahtarıdır.
!wget -O /content/data.zip https://universe.roboflow.com/ds/nmGz9ajNlF?key=oGO5G6pgvq

Bu hücre, bir önceki adımda indirilen data.zip dosyasının içeriğini /content/custom_data adlı bir klasöre çıkarır.
-q: "Quiet" modu, çıkarma işlemi sırasında detaylı dosya listesi çıktısını gizler, sadece hatalar gösterilir.
-d /content/custom_data: Çıkarılan dosyaların kaydedileceği hedef dizini belirtir.


In [ ]:
# 3. Veri Kümesi Çıkarma
# İndirilen '/content/data.zip' dosyasını '/content/custom_data' klasörüne çıkarır.
# '-q' (quiet) seçeneği, çıkarma işlemi sırasında detaylı çıktıları gizler.
# '-d /content/custom_data' seçeneği, çıktı dosyalarının kaydedileceği dizini belirtir.
!unzip -q /content/data.zip -d /content/custom_data

Bu uzun kod hücresi, indirilen ve çıkarılan veri kümesini YOLO eğitimi için uygun bir formata getirmek üzere birkaç önemli işlem yapar:
1. **Ayarlar:** Giriş (INPUT_DATA_DIR), çıkış (OUTPUT_DATA_DIR) dizinleri ve veri kümesinin eğitim, doğrulama ve test setlerine bölünme oranları (TRAIN_RATIO, VALID_RATIO) tanımlanır.
2. **Dosyaları Toplama:** Orijinal veri kümesindeki (/content/custom_data) train, valid, test klasörlerindeki tüm resim (.jpg, .png vb.) ve bunlara karşılık gelen etiket (.txt) dosyalarının yollarını toplar. Eşleşmeyen dosyalar için uyarı verir.
3. **Karıştırma ve Bölme:** Toplanan resim-etiket çiftlerini rastgele karıştırır. Ardından, tanımlanan oranlara göre eğitim, doğrulama ve test setlerine ayırır.
4. **Yeni Klasör Yapısı Oluşturma:** /content/rebalanced_data altında yeni train/images, train/labels, valid/images, valid/labels, test/images, test/labels klasörlerini oluşturur. Eğer bu klasörler zaten varsa, temizler.
5. **Dosyaları Kopyalama:** Bölünen dosyaları yeni oluşturulan klasörlere kopyalar.
6. **data.yaml Oluşturma:** YOLO eğitiminde kullanılacak olan data.yaml yapılandırma dosyasını oluşturur. Bu dosya, eğitim, doğrulama ve test setlerinin yollarını, sınıf sayısını (nc) ve sınıf isimlerini (names) içerir. Sınıf bilgilerini orijinal data.yaml dosyasından (varsa) almaya çalışır.

In [ ]:
# 4. Veri Kümesi Yeniden Dengeleme ve Organizasyon
# Bu betik, '/content/custom_data' içindeki mevcut veri kümesini (train, valid, test klasörlerinden)
# alır, tüm resim ve etiketleri birleştirir, karıştırır ve ardından belirtilen oranlarda
# '/content/rebalanced_data' altında yeni train, valid, test klasörlerine böler.
# Ayrıca, yeni oluşturulan veri kümesi için bir 'data.yaml' dosyası da oluşturur.

import os
import shutil # Dosya ve klasör işlemleri için (kopyalama, silme vb.)
import random # Dosyaları karıştırmak için
from pathlib import Path # Modern dosya yolu işlemleri için
import yaml # YAML dosyalarını okumak ve yazmak için (PyYAML kütüphanesi)

# --- Ayarlar ---
INPUT_DATA_DIR = Path("/content/custom_data") # Orijinal veri kümesinin bulunduğu ana klasör
OUTPUT_DATA_DIR = Path("/content/rebalanced_data") # Yeniden dengelenmiş veri kümesinin kaydedileceği klasör
TRAIN_RATIO = 0.8  # Eğitim seti için toplam veri kümesinin oranı (%80)
VALID_RATIO = 0.11 # Doğrulama seti için toplam veri kümesinin oranı (%11)
# TEST_RATIO, (1 - TRAIN_RATIO - VALID_RATIO) olarak otomatik hesaplanacak (%9)

# --- 1. Tüm resim ve etiket dosyalarını topla ---
all_image_paths = [] # Tüm resim dosyalarının yollarını tutacak liste
all_label_paths = [] # Tüm etiket dosyalarının yollarını tutacak liste

print("Mevcut veri setindeki dosyalar toplanıyor...")
# Orijinal veri kümesindeki 'train', 'valid', 'test' klasörlerini dolaş
for split_folder_name in ["train", "valid", "test"]:
    image_folder = INPUT_DATA_DIR / split_folder_name / "images"
    label_folder = INPUT_DATA_DIR / split_folder_name / "labels"

    # Klasörlerin var olup olmadığını kontrol et
    if not image_folder.exists():
        print(f"Uyarı: {image_folder} bulunamadı, bu bölüm atlanıyor.")
        continue # Bu bölümü atla ve sonraki split_folder_name'e geç
    if not label_folder.exists():
        print(f"Uyarı: {label_folder} bulunamadı, bu bölüm atlanıyor.")
        continue

    # Resim klasöründeki tüm dosyaları al (uzantıya bakılmaksızın)
    for img_file_path in image_folder.glob("*.*"): # .jpg, .png, vb. tüm resim formatlarını alır
        # İlgili etiket dosyasının adını oluştur (örneğin, image1.jpg -> image1.txt)
        label_file_name = img_file_path.stem + ".txt" # .stem dosya adını uzantısız verir
        label_file_path = label_folder / label_file_name

        # Etiket dosyasının var olup olmadığını kontrol et
        if label_file_path.exists():
            all_image_paths.append(img_file_path)
            all_label_paths.append(label_file_path)
        else:
            print(f"Uyarı: {img_file_path} için etiket dosyası ({label_file_path}) bulunamadı. Bu resim atlanıyor.")

# Toplanan dosya olup olmadığını kontrol et
if not all_image_paths:
    print("Hiç resim dosyası bulunamadı. Lütfen INPUT_DATA_DIR yolunu kontrol edin.")
    # exit() # Colab'da script'i durdurmak için daha iyi bir yol, bir hata yükseltmek olabilir
    raise SystemExit("Veri toplama başarısız, resim bulunamadı.")

print(f"Toplam {len(all_image_paths)} adet resim ve etiket çifti bulundu.")

# --- 2. Dosyaları eşleştir ve karıştır ---
# Resim ve etiket yollarını (Path nesneleri olarak) çiftler halinde sakla
file_pairs = list(zip(all_image_paths, all_label_paths))
random.shuffle(file_pairs) # Listeyi yerinde (in-place) karıştırır, böylece rastgele bir dağılım elde edilir

# --- 3. Veri setini oranlara göre böl ---
total_files = len(file_pairs)
train_count = int(total_files * TRAIN_RATIO) # Eğitim seti için dosya sayısı
valid_count = int(total_files * VALID_RATIO) # Doğrulama seti için dosya sayısı
test_count = total_files - train_count - valid_count # Kalanlar test seti için

# Karıştırılmış listeden dilimleyerek setleri oluştur
train_files = file_pairs[:train_count]
valid_files = file_pairs[train_count : train_count + valid_count]
test_files = file_pairs[train_count + valid_count :]

print(f"\nYeni dağılım:")
print(f"Train: {len(train_files)} dosya ({len(train_files)/total_files*100:.1f}%)")
print(f"Valid: {len(valid_files)} dosya ({len(valid_files)/total_files*100:.1f}%)")
print(f"Test:  {len(test_files)} dosya ({len(test_files)/total_files*100:.1f}%)")

# --- 4. Yeni klasör yapısını oluştur ve dosyaları kopyala ---
# Eğer çıktı klasörü zaten varsa, içeriğini sil (temiz bir başlangıç için)
if OUTPUT_DATA_DIR.exists():
    print(f"\nUyarı: {OUTPUT_DATA_DIR} klasörü zaten mevcut. İçeriği silinecek.")
    shutil.rmtree(OUTPUT_DATA_DIR) # Klasörü ve içeriğini rekürsif olarak siler
OUTPUT_DATA_DIR.mkdir(parents=True, exist_ok=True) # Ana klasörü ve gerekirse üst klasörleri oluşturur

# Dosyaları ilgili train/valid/test klasörlerine kopyalamak için yardımcı fonksiyon
def create_split_folders_and_copy(file_list, split_name):
    img_output_folder = OUTPUT_DATA_DIR / split_name / "images"
    lbl_output_folder = OUTPUT_DATA_DIR / split_name / "labels"

    # Hedef klasörleri oluştur (images ve labels)
    img_output_folder.mkdir(parents=True, exist_ok=True)
    lbl_output_folder.mkdir(parents=True, exist_ok=True)

    print(f"\n{split_name} dosyaları kopyalanıyor...")
    copied_count = 0
    for img_path, lbl_path in file_list:
        try:
            # shutil.copy2, dosya verilerini ve mümkünse meta verilerini kopyalar
            shutil.copy2(img_path, img_output_folder / img_path.name)
            shutil.copy2(lbl_path, lbl_output_folder / lbl_path.name)
            copied_count += 1
        except Exception as e:
            print(f"Hata: {img_path} veya {lbl_path} kopyalanamadı. {e}")
    print(f"{split_name} için {copied_count} dosya çifti kopyalandı.")

# Fonksiyonu her bir set için çağır
create_split_folders_and_copy(train_files, "train")
create_split_folders_and_copy(valid_files, "valid")
create_split_folders_and_copy(test_files, "test")

# --- 5. Yeni data.yaml dosyasını oluştur ---
# Orijinal veri kümesindeki 'data.yaml' dosyasını bulup sınıf bilgilerini (names, nc) almayı dene
original_yaml_path = INPUT_DATA_DIR / "data.yaml"
names = [] # Sınıf isimleri listesi
nc = 0     # Sınıf sayısı (number of classes)

if original_yaml_path.exists():
    try:
        with open(original_yaml_path, 'r', encoding='utf-8') as f: # encoding eklendi
            data_yaml_content = yaml.safe_load(f)
        names = data_yaml_content.get('names', []) # 'names' anahtarı varsa al, yoksa boş liste
        nc = data_yaml_content.get('nc', len(names)) # 'nc' varsa al, yoksa 'names' listesinin uzunluğunu kullan

        # Orijinal YAML'da bu anahtarların olup olmadığını kontrol et
        if not names and 'names' not in data_yaml_content:
            print("Uyarı: Orijinal data.yaml dosyasında 'names' anahtarı bulunamadı.")
        if nc == 0 and 'nc' not in data_yaml_content and not names:
             print("Uyarı: Orijinal data.yaml dosyasında 'nc' anahtarı bulunamadı ve 'names' de boş.")

    except Exception as e:
        print(f"Uyarı: Orijinal data.yaml ({original_yaml_path}) okunamadı. Hata: {e}")
        print("Yeni data.yaml dosyası için sınıf bilgileri manuel olarak girilmeli veya varsayılanlar kullanılacak.")
else:
    print(f"Uyarı: Orijinal data.yaml ({original_yaml_path}) bulunamadı.")
    print("Yeni data.yaml dosyası için sınıf bilgileri manuel olarak girilmeli veya varsayılanlar kullanılacak.")

# Sınıf bilgileri hala eksikse ek kontroller yap
if not names: # Eğer names hala boşsa (okunamadı veya orijinalde yoktu)
    print("Sınıf isimleri (names) alınamadı. Yeni data.yaml eksik olabilir. 'classes.txt' kontrol edilebilir.")
if nc == 0 and names: # nc okunamadı ama names listesi dolu (örneğin, eski yaml'da nc yoktu)
    nc = len(names)
    print(f"Sınıf sayısı (nc), 'names' listesinden {nc} olarak ayarlandı.")
elif nc == 0 and not names: # Hem nc hem de names alınamadı
    print("Kritik Uyarı: Sınıf sayısı (nc) ve sınıf isimleri (names) belirlenemedi. YOLO eğitimi başarısız olabilir.")
    print("Lütfen '/content/rebalanced_data/data.yaml' dosyasını manuel olarak düzenleyin.")

# Yeni data.yaml içeriğini oluştur
new_data_yaml_content = {
    'path': str(OUTPUT_DATA_DIR.resolve()), # Mutlak yol, YOLO için genellikle daha iyidir
    'train': 'train/images', # 'path'e göre göreceli yol
    'val': 'valid/images',   # YOLO genellikle 'val' kullanır, 'valid' de kabul edilebilir
    'test': 'test/images',   # İsteğe bağlı, test seti için
    'nc': nc,
    'names': names
}

new_yaml_path = OUTPUT_DATA_DIR / "data.yaml"
try:
    with open(new_yaml_path, 'w', encoding='utf-8') as f: # encoding eklendi
        # sort_keys=False: YAML'daki anahtar sırasını korur
        # default_flow_style=None: Daha okunabilir blok stili YAML üretir
        yaml.dump(new_data_yaml_content, f, sort_keys=False, default_flow_style=None, allow_unicode=True)
    print(f"\nYeni data.yaml dosyası şuraya oluşturuldu: {new_yaml_path}")
    print("data.yaml içeriği:")
    print(yaml.dump(new_data_yaml_content, sort_keys=False, default_flow_style=None, allow_unicode=True))
except Exception as e:
    print(f"Hata: Yeni data.yaml dosyası ({new_yaml_path}) oluşturulamadı. Hata: {e}")


print("\nVeri kümesi yeniden dengeleme ve organizasyon işlemi tamamlandı!")
print(f"Yeniden dengelenmiş veri seti '{OUTPUT_DATA_DIR}' klasörüne kaydedildi.")

Bu hücre, YOLOv11 (ve önceki versiyonları) gibi nesne algılama modellerini eğitmek ve kullanmak için gerekli olan ultralytics kütüphanesini pip kullanarak kurar.

In [ ]:
# 5. Ultralytics Kurulumu
# Ultralytics, YOLOv5, YOLOv8 gibi modelleri ve eğitim araçlarını içeren bir Python kütüphanesidir.
# '!pip install ultralytics' komutu ile en son sürümü PyPI'dan indirip kurar.
!pip install ultralytics

Bu hücre, ultralytics kütüphanesini kullanarak YOLO modelini eğitir:
1. **GPU Kontrolü:** Tekrar GPU kullanılabilirliğini kontrol eder ve eğitim için device değişkenini ayarlar.
2. **Model Yükleme:** YOLO(model_name) ile bir YOLO modeli yükler. model_name (örneğin, yolov8n.pt) önceden eğitilmiş bir modelin adı olabilir. Ultralytics, bu model mevcut değilse genellikle indirir.
3. **Veri Yolu Kontrolü:** Bir önceki adımda oluşturulan /content/rebalanced_data/data.yaml dosyasının varlığını kontrol eder.
4. **Model Eğitimi (model.train):**
 1. data: Eğitim için kullanılacak data.yaml dosyasının yolu.
 2. epochs: Eğitim döngüsü sayısı.
 3. batch: Her iterasyonda işlenecek resim sayısı. GPU belleğine göre ayarlanmalıdır.
 4. imgsz: Modele verilecek resimlerin boyutu.
 5. cache: Veri yüklemesini hızlandırmak için resimlerin diskte veya RAM'de önbelleğe alınmasını sağlar.
 6. workers: Veri yükleme işlemleri için kullanılacak paralel işlemci sayısı.
 7. project: Eğitim sonuçlarının (ağırlıklar, loglar, grafikler) kaydedileceği ana klasör.
 8. name: Bu özel eğitim çalıştırması için alt klasör adı.
 9. exist_ok: Aynı name ile bir klasör varsa üzerine yazmaya izin verir.
 10. optimizer: Kullanılacak optimizasyon algoritması (auto, SGD, Adam vb.).
 11. amp: Otomatik Karışık Hassasiyet (AMP) kullanımı. Eğitimi hızlandırabilir ve VRAM kullanımını azaltabilir.
 12. device: Eğitim için kullanılacak cihaz (cuda:0 veya cpu).
5. **Doğrulama (model_best.val):** Eğitim tamamlandıktan sonra, en iyi performansı gösteren model ağırlıkları (best.pt) yüklenir ve doğrulama seti üzerinde performansı (mAP gibi metrikler) değerlendirilir.
6. **Hata Yönetimi:** Eğitim sırasında "out of memory" gibi hatalar oluşursa, kullanıcıya bilgi verir ve GPU durumunu gösterir.

In [ ]:
# 7. YOLO Model Eğitimi

from ultralytics import YOLO
import torch
import subprocess # nvidia-smi gibi harici komutları çalıştırmak için
from pathlib import Path # Dosya yolu işlemleri için

# GPU kullanılabilirliğini kontrol et
if torch.cuda.is_available():
    device = torch.device("cuda:0") # İlk kullanılabilir GPU'yu seç
    print(f"GPU kullanılabilir: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("GPU bulunamadı, CPU üzerinde çalışılacak.")

# 1. Modeli yükle
# 'yolo11n.pt' (veya 'yolov8n.pt' gibi) önceden eğitilmiş bir model olabilir veya özel bir model adı olabilir.
# Eğer bu dosya mevcut değilse veya bir YAML yapılandırma dosyası değilse, Ultralytics
# genellikle bu isimde bir modeli (varsa) indirir veya sıfırdan oluşturur (eğer bir .yaml ise).
model_name = 'yolov8n.pt' # Kullanıcı 'yolo11n.pt' yazmıştı, yaygın bir model olan 'yolov8n.pt' ile değiştirildi. İstenirse eski haline getirilebilir.
try:
    model = YOLO(model_name) # Modeli yükle veya başlat
    print(f"'{model_name}' modeli başarıyla yüklendi/başlatıldı.")
except Exception as e:
    print(f"Hata: '{model_name}' modeli yüklenemedi. {e}")
    print("Lütfen geçerli bir model adı (.pt dosyası), YAML yapılandırma dosyası (.yaml) veya resmi bir YOLO modeli adı (örn: yolov8n) sağladığınızdan emin olun.")
    raise SystemExit("Model yükleme başarısız.")

# Veri YAML dosyasının yolu (bir önceki adımda oluşturulan, yeniden dengelenmiş veri için)
data_yaml_path = '/content/rebalanced_data/data.yaml'
if not Path(data_yaml_path).exists():
    print(f"Kritik Hata: Veri yapılandırma dosyası bulunamadı: {data_yaml_path}")
    print("Lütfen bir önceki ('Veri Kümesi Yeniden Dengeleme') hücrenin başarıyla çalıştığından emin olun.")
    raise SystemExit("data.yaml bulunamadı.")

# 2. Modeli eğit
try:
    print(f"Eğitim başlatılıyor... Veri dosyası: {data_yaml_path}")
    results = model.train(
        data=data_yaml_path,  # Veri kümesi yapılandırma dosyasının yolu
        epochs=30,            # Eğitim yapılacak epoch sayısı
        batch=50,             # Batch boyutu (GPU belleğine göre ayarlanabilir)
        imgsz=416,            # Giriş resimlerinin boyutu (örn: 640, 416)
        cache='disk',         # Veri yüklemesini hızlandırmak için resimleri diskte önbelleğe al ('ram' veya False da olabilir)
        workers=2,            # Veri yükleme için kullanılacak işçi (worker) sayısı (CPU çekirdek sayısına göre)
        project='runs/detect',# Eğitim sonuçlarının kaydedileceği ana klasör
        name='train_experiment_epoch30', # Bu eğitim çalıştırması için özel isim (klasör adı olacak)
        exist_ok=True,        # Eğer 'name' ile aynı isimde bir klasör varsa üzerine yazmaya izin ver
        optimizer='auto',     # Optimizasyon algoritması ('SGD', 'Adam', 'AdamW', 'auto')
        amp=True,             # Otomatik Karışık Hassasiyet (Automatic Mixed Precision) - eğitimi hızlandırır ve VRAM kullanımını azaltır
        device=device,        # Eğitim için kullanılacak cihaz (CPU veya GPU)
    )
    print("Eğitim tamamlandı!")

    # `results` objesi, eğitimin bittiği epoch'un modelini ve save_dir'i içerebilir,
    # ancak en iyi model genellikle `model.trainer.best` ile veya save_dir içinden alınır.
    save_dir = Path(model.trainer.save_dir if hasattr(model, 'trainer') and model.trainer else results.save_dir)
    print(f"Sonuçlar şuraya kaydedildi: {save_dir}")

    # Eğitim sonrası en iyi modeli kullanarak doğrulama (validation)
    print("\nEğitilmiş en iyi model ile doğrulama yapılıyor...")

    best_model_path_str = None
    if hasattr(model, 'trainer') and model.trainer and hasattr(model.trainer, 'best') and model.trainer.best:
        best_model_path_str = str(model.trainer.best) # Path objesine dönüştürmeden önce string olduğundan emin olalım

    if best_model_path_str and Path(best_model_path_str).exists():
        print(f"En iyi model yükleniyor: {best_model_path_str}")
        model_best = YOLO(best_model_path_str) # En iyi .pt dosyasını yükle
        metrics = model_best.val(data=data_yaml_path, split='val') # data.yaml'daki 'val' setini ve doğru yaml'ı kullanır
        print("Doğrulama metrikleri (mAP50-95):", metrics.box.map)
        print("Doğrulama metrikleri (mAP50):", metrics.box.map50)
        print("Doğrulama metrikleri (mAP75):", metrics.box.map75)
    else:
        # Alternatif olarak, save_dir içindeki weights/best.pt'yi arayabiliriz
        best_model_path_alt = save_dir / 'weights' / 'best.pt'
        if best_model_path_alt.exists():
            print(f"En iyi model (alternatif yol) yükleniyor: {best_model_path_alt}")
            model_best = YOLO(str(best_model_path_alt))
            metrics = model_best.val(data=data_yaml_path, split='val') # data.yaml'daki 'val' setini ve doğru yaml'ı kullanır
            print("Doğrulama metrikleri (mAP50-95):", metrics.box.map)
            print("Doğrulama metrikleri (mAP50):", metrics.box.map50)
            print("Doğrulama metrikleri (mAP75):", metrics.box.map75)
        else:
            print(f"Uyarı: En iyi model ne '{best_model_path_str}' ne de '{best_model_path_alt}' yolunda bulunamadı.")
            print(f"Lütfen '{save_dir / 'weights' / 'best.pt'}' yolunu manuel kontrol edin.")
            print("Doğrulama adımı atlanıyor.")

except Exception as e:
    print(f"Eğitim veya doğrulama sırasında bir hata oluştu: {e}")
    if "out of memory" in str(e).lower() and torch.cuda.is_available():
        print("CUDA Out of Memory hatası! Batch boyutunu ('batch') düşürmeyi veya resim boyutunu ('imgsz') küçültmeyi deneyin.")
    if torch.cuda.is_available(): # Hata durumunda GPU durumunu tekrar kontrol et
        print("\nGPU Durumu (Hata Sonrası):")
        try:
            # nvidia-smi komutunu çalıştır ve çıktısını al
            output = subprocess.check_output(['nvidia-smi'], universal_newlines=True)
            print(output)
        except Exception as smi_e:
            print(f"nvidia-smi çalıştırılamadı: {smi_e}")
    raise # Hatanın tekrar yükseltilmesi, Colab'ın hücreyi hatalı olarak işaretlemesini sağlar

In [ ]:
# 8. Eğitilmiş Modelin Test Seti Üzerinde Değerlendirilmesi ve Sonuçların Kaydedilmesi

from ultralytics import YOLO
from pathlib import Path
import os
import shutil # Çıktı klasörünü yönetmek için

# --- Ayarlar ---
# Bir önceki eğitim adımında (model.train) 'project' ve 'name' parametreleriyle eşleşmeli
EXPERIMENT_PARENT_DIR = Path("/content/runs/detect")
EXPERIMENT_NAME = "train_experiment_epoch30" # Eğitimin 'name' parametresi

# Test edilecek modelin yolu (eğitim sonrası en iyi model)
# Bu yol, eğitim çıktılarındaki best.pt dosyasını işaret etmelidir.
BEST_MODEL_PATH = EXPERIMENT_PARENT_DIR / EXPERIMENT_NAME / "weights" / "best.pt"

# Yeniden dengelenmiş veri setinin YAML dosyası
# Bu dosya, test seti yolunu ('test': 'test/images') içermelidir.
DATA_YAML_PATH = Path("/content/rebalanced_data/data.yaml")

# Test sonuçlarının kaydedileceği proje ve isim
# Bu, eğitimden farklı bir klasörde sonuçları saklamak için kullanışlıdır.
TEST_PROJECT_DIR = Path("/content/runs/test") # Ana test çıktı klasörü
TEST_RUN_NAME = "test_on_rebalanced_data"    # Bu test çalıştırması için özel isim

# Test için kullanılacak resim boyutu (eğitimdeki imgsz ile aynı veya uyumlu olmalı)
TEST_IMG_SIZE = 416 # Eğitimde kullandığınız imgsz değeriyle aynı yapın

# Test için batch boyutu
TEST_BATCH_SIZE = 48 # GPU belleğinize göre ayarlayabilirsiniz

# --- 1. Gerekli Yolların ve Modelin Varlığını Kontrol Et ---
if not BEST_MODEL_PATH.exists():
    print(f"HATA: En iyi model dosyası bulunamadı: {BEST_MODEL_PATH}")
    print("Lütfen bir önceki eğitim hücresinin başarıyla tamamlandığından ve 'best.pt' dosyasının oluştuğundan emin olun.")
    # Colab'da script'i durdurmak için:
    raise SystemExit("Test edilecek model bulunamadı.")

if not DATA_YAML_PATH.exists():
    print(f"HATA: Veri YAML dosyası bulunamadı: {DATA_YAML_PATH}")
    print("Lütfen veri kümesi hazırlama adımlarının doğru tamamlandığından emin olun.")
    raise SystemExit("data.yaml bulunamadı.")

# --- 2. Test Çıktı Klasörünü Hazırla (İsteğe Bağlı ama Önerilir) ---
# Eğer daha önce aynı isimle bir test çalıştırıldıysa, eski sonuçların üzerine yazılmasını önlemek
# veya temiz bir başlangıç yapmak için eski klasörü silebiliriz.
# Veya `exist_ok=True` parametresini .val() veya .predict() içinde kullanabiliriz.
# Bu örnekte, YOLO'nun kendi 'exist_ok' mekanizmasına güveneceğiz.

# --- 3. Modeli Yükle ---
print(f"Test için model yükleniyor: {BEST_MODEL_PATH}")
try:
    model = YOLO(str(BEST_MODEL_PATH))
    print("Model başarıyla yüklendi.")
except Exception as e:
    print(f"HATA: Model yüklenirken bir sorun oluştu: {e}")
    raise SystemExit("Model yükleme başarısız.")

# --- 4. Modeli Test Seti Üzerinde Değerlendir (metrics.val()) ---
# .val() metodu, data.yaml dosyasında belirtilen 'test' (veya 'val') bölümünü kullanarak
# mAP gibi standart metrikleri hesaplar.
# Eğer data.yaml dosyanızda 'test' anahtarı doğru bir şekilde test verilerinizi gösteriyorsa,
# split='test' parametresini kullanabilirsiniz. Eğer 'test' anahtarı yoksa veya
# doğrulama (validation) setini test amacıyla kullanmak istiyorsanız split='val' kullanın.
# data.yaml dosyamızda 'test' yolunun '/content/rebalanced_data/test/images' olduğunu varsayıyoruz.

print(f"\nModel test seti üzerinde değerlendiriliyor (metrics.val())...")
print(f"Veri YAML: {DATA_YAML_PATH}")
print(f"Çıktılar şuraya kaydedilecek: {TEST_PROJECT_DIR / TEST_RUN_NAME}")

try:
    metrics = model.val(
        data=str(DATA_YAML_PATH),
        split='test',  # data.yaml içindeki 'test' anahtarını kullanır
        imgsz=TEST_IMG_SIZE,
        batch=TEST_BATCH_SIZE,
        project=str(TEST_PROJECT_DIR), # Sonuçların kaydedileceği ana klasör
        name=TEST_RUN_NAME,         # Bu test çalıştırması için özel alt klasör adı
        exist_ok=True,              # Eğer aynı isimde klasör varsa üzerine yaz
        save_json=True,             # COCO formatında JSON sonuçları kaydeder (mAP hesaplaması için)
        save_hybrid=True,           # Hem etiketleri hem de tahminleri içeren resimleri kaydeder (görsel inceleme için)
        conf=0.25,                  # Tahminler için minimum güven eşiği (varsayılan)
        iou=0.45                    # NMS için IoU eşiği (varsayılan)
    )
    print("\nTest Değerlendirme Metrikleri:")
    if hasattr(metrics, 'box'):
        print(f"  mAP50-95: {metrics.box.map:.4f}")
        print(f"  mAP50:    {metrics.box.map50:.4f}")
        print(f"  mAP75:    {metrics.box.map75:.4f}")
        # Sınıf bazlı mAP değerlerini de yazdırabiliriz (eğer varsa)
        if hasattr(metrics.box, 'maps') and metrics.box.maps is not None:
            for i, class_name in enumerate(model.names):
                 if i < len(metrics.box.maps): # maps dizisinin sınırları içinde mi kontrolü
                    print(f"  mAP50-95 ({class_name}): {metrics.box.maps[i]:.4f}")
    else:
        print("Detaylı kutu metrikleri bulunamadı. Lütfen 'metrics' objesini kontrol edin.")

    print(f"\nDeğerlendirme sonuçları ve görseller şuraya kaydedildi: {TEST_PROJECT_DIR / TEST_RUN_NAME}")

except Exception as e:
    print(f"HATA: Model değerlendirme (model.val()) sırasında bir sorun oluştu: {e}")
    raise

# --- 5. Test Seti Üzerinde Tahminler Yap ve Sonuçları Kaydet (model.predict()) ---
# model.predict() metodu, test setindeki her resim için tahminler yapar ve
# bu tahminleri (sınırlayıcı kutular, etiketler, güven skorları ile birlikte) resimlerin üzerine çizebilir.

print(f"\nTest seti üzerinde tahminler yapılıyor (model.predict())...")
# Tahminlerin kaydedileceği klasör, .val() ile aynı yapıda olacak şekilde ayarlanır.
# YOLO, project/name yapısını kendi yönetir. predict() metodu içinde de benzer parametreler vardır.
# Ancak, predict() doğrudan metrik hesaplamaz, daha çok görsel çıktı ve ham tahminler üretir.

# Çıktıların kaydedileceği spesifik bir alt klasör belirleyelim
PREDICT_SAVE_DIR = TEST_PROJECT_DIR / TEST_RUN_NAME / "predictions"
PREDICT_SAVE_DIR.mkdir(parents=True, exist_ok=True) # Klasörü oluştur

# data.yaml dosyasından test resimlerinin yolunu alalım
import yaml
with open(DATA_YAML_PATH, 'r') as f:
    data_config = yaml.safe_load(f)
# data.yaml'daki 'path' ve 'test' anahtarlarını birleştirerek tam yolu oluştur
# Path(data_config['path']) mutlak bir yol olmalı (önceki hücrede öyle ayarlandı)
test_images_path_pattern = Path(data_config['path']) / data_config['test']

if not test_images_path_pattern.exists():
     # Eğer path göreceli ise ve data.yaml ile aynı dizindeyse:
     alternative_test_path = DATA_YAML_PATH.parent / data_config['test']
     if alternative_test_path.exists():
         test_images_path_pattern = alternative_test_path
     else:
        print(f"HATA: Test resimleri yolu bulunamadı: {test_images_path_pattern} veya {alternative_test_path}")
        raise SystemExit("Test resim yolu hatalı.")


print(f"Tahmin yapılacak resimler: {test_images_path_pattern}")

try:
    results = model.predict(
        source=str(test_images_path_pattern), # Test resimlerinin bulunduğu klasör veya glob deseni
        imgsz=TEST_IMG_SIZE,
        conf=0.25,        # Tahminler için minimum güven skoru
        iou=0.45,         # NMS için IoU eşiği
        save=True,        # Tahminlerin çizildiği resimleri kaydeder
        save_txt=True,    # Tahminleri YOLO formatında .txt dosyalarına kaydeder
        save_conf=True,   # .txt dosyalarına güven skorlarını da ekler
        project=str(TEST_PROJECT_DIR), # Ana çıktı klasörü
        name=TEST_RUN_NAME + "_preds", # Tahminler için ayrı bir alt klasör adı
        exist_ok=True,
        # line_width veya line_thickness gibi çizim parametreleri de eklenebilir
    )
    # results bir jeneratördür, her bir resim için sonuçları işleyebilirsiniz.
    # Bu döngü, tahminlerin yapılmasını tetikler ve sonuçların kaydedilmesini sağlar.
    processed_count = 0
    for r in results:
        processed_count +=1
        # r.path, r.names, r.boxes vb. özelliklere erişebilirsiniz.
        # Kaydetme işlemi yukarıdaki 'save=True' ile zaten yapılıyor.
        pass # Sadece jeneratörü tüketmek için

    print(f"\n{processed_count} resim için tahmin yapıldı ve sonuçlar kaydedildi.")
    print(f"Tahmin görselleri ve .txt dosyaları şuraya kaydedildi: {TEST_PROJECT_DIR / (TEST_RUN_NAME + '_preds')}")

except Exception as e:
    print(f"HATA: Model tahmini (model.predict()) sırasında bir sorun oluştu: {e}")
    raise

print("\nTest ve tahmin işlemleri tamamlandı!")

Bu hücre, bir önceki eğitim adımında oluşturulan tüm önemli çıktıları (en iyi model ağırlıkları, eğitim logları, grafikler vb.) toplar ve tek bir .zip dosyası olarak paketler.
1. **Ayarlar:** EXPERIMENT_NAME (bir önceki hücredeki model.train fonksiyonundaki name parametresiyle aynı olmalıdır), kaynak ve hedef dizinler, oluşturulacak geçici klasör adı ve son zip dosyasının adı tanımlanır.
2. **Kaynak Kontrolü:** SOURCE_RUN_DIR (yani /content/runs/detect/train_experiment_epoch30) klasörünün var olup olmadığını kontrol eder.
3. **Hedef Klasör Hazırlığı:** /content/my_yolo_model_outputs gibi geçici bir klasör oluşturur (varsa silip yeniden oluşturur).
4. **En İyi Modelin Kopyalanması:** SOURCE_RUN_DIR/weights/best.pt dosyasını, hedef klasöre best_trained_model.pt adıyla kopyalar.
5. **Tüm Çalıştırma Klasörünün Kopyalanması:** Tüm SOURCE_RUN_DIR (örneğin, train_experiment_epoch30) klasörünü, tüm içeriğiyle (loglar, grafikler vb.) hedef klasörün içine kopyalar.
6. **Sıkıştırma:** Hedef klasörü (my_yolo_model_outputs) bir zip dosyası (my_trained_yolo_model_archive.zip) olarak /content dizinine sıkıştırır.

In [ ]:
# 9. Sonuçları Arşivleme (Genişletilmiş Kapsam)
# Bu betik, eğitim çıktılarını, test çıktılarını ve potansiyel bir doğrulama çalıştırmasını
# toplayıp tek bir zip dosyası olarak paketler.

import os
import shutil # Dosya ve klasör işlemleri için
from pathlib import Path # Modern dosya yolu işlemleri için

# --- Ayarlarınızı Buradan Yapılandırın ---
# Eğitim betiğinizde model.train() içindeki 'name' parametresiyle aynı olmalı
TRAIN_EXPERIMENT_NAME = "train_experiment_epoch30" # Eğitimin 'name' parametresi

# Test çalıştırmasının adı (bir önceki test hücresindeki TEST_RUN_NAME ile aynı olmalı)
TEST_RUN_NAME_VAL_METRICS = "test_on_rebalanced_data" # model.val() için kullanılan name
TEST_RUN_NAME_PREDICTIONS = "test_on_rebalanced_data_preds" # model.predict() için kullanılan name

# Potansiyel ayrı bir doğrulama çalıştırmasının adı (varsa)
# Eğer eğitim sırasında otomatik doğrulama yapılıyorsa bu boş bırakılabilir veya None yapılabilir.
# Eğer model.val() komutunu ayrı bir 'name' ile (örn: 'val') çalıştırdıysanız buraya onu yazın.
SEPARATE_VAL_RUN_NAME = "val" # Örnek bir isim, eğer yoksa None yapın veya boş bırakın.

# Kaynak dizinler
SOURCE_DETECT_PARENT_DIR = Path("/content/runs/detect") # Eğitim ve potansiyel doğrulama çıktıları
SOURCE_TEST_PARENT_DIR = Path("/content/runs/test")    # Test çıktıları

# Hedef arşiv ayarları
TARGET_ARCHIVE_PARENT_DIR = Path("/content")
TARGET_OUTPUT_FOLDER_NAME = "my_complete_yolo_run_outputs" # Zip açıldığında görünecek ana klasör adı
ZIP_FILE_BASENAME = "my_complete_yolo_archive" # Oluşturulacak zip dosyasının adı

# --- Yol Tanımlamaları ---
SOURCE_TRAIN_RUN_DIR = SOURCE_DETECT_PARENT_DIR / TRAIN_EXPERIMENT_NAME
SOURCE_TEST_VAL_METRICS_DIR = SOURCE_TEST_PARENT_DIR / TEST_RUN_NAME_VAL_METRICS
SOURCE_TEST_PREDICTIONS_DIR = SOURCE_TEST_PARENT_DIR / TEST_RUN_NAME_PREDICTIONS

SOURCE_SEPARATE_VAL_DIR = None
if SEPARATE_VAL_RUN_NAME: # Eğer bir isim verilmişse
    SOURCE_SEPARATE_VAL_DIR = SOURCE_DETECT_PARENT_DIR / SEPARATE_VAL_RUN_NAME

TARGET_OUTPUT_DIR = TARGET_ARCHIVE_PARENT_DIR / TARGET_OUTPUT_FOLDER_NAME
FINAL_ZIP_PATH = TARGET_ARCHIVE_PARENT_DIR / f"{ZIP_FILE_BASENAME}.zip"

def copy_directory_if_exists(src_dir_path, dest_parent_path, new_folder_name=None):
    """
    Kaynak dizini var ise hedefteki bir üst dizinin içine kopyalar.
    Eğer new_folder_name verilirse, kaynak dizin bu isimle kopyalanır.
    """
    if src_dir_path and src_dir_path.is_dir():
        target_name = new_folder_name if new_folder_name else src_dir_path.name
        dest_path = dest_parent_path / target_name
        print(f"'{src_dir_path.name}' klasörü kopyalanıyor:")
        print(f"  Kaynak: {src_dir_path}")
        print(f"  Hedef:  {dest_path}")
        try:
            shutil.copytree(src_dir_path, dest_path)
            print(f"'{src_dir_path.name}' başarıyla kopyalandı.\n")
            return True
        except FileExistsError:
            print(f"Hata: Hedef klasör '{dest_path}' zaten mevcut. Bu bir sorun olmamalıydı.")
            return False # Veya üzerine yazma stratejisi eklenebilir
        except Exception as e:
            print(f"Hata: '{src_dir_path.name}' kopyalanamadı. {e}\n")
            return False
    else:
        if src_dir_path: # Yol tanımlı ama klasör değilse
            print(f"UYARI: Kaynak '{src_dir_path}' bulunamadı veya bir klasör değil. Kopyalama atlanıyor.\n")
        # src_dir_path None ise (örn: SEPARATE_VAL_RUN_NAME verilmemişse) sessiz kal
        return False


def collect_and_zip_all_outputs():
    print("Tüm model çıktılarını toplama ve sıkıştırma işlemi başlatılıyor...\n")

    # 1. Hedef çıktı klasörünü oluştur (varsa üzerine yazılacak şekilde temizle)
    print(f"Hedef çıktı klasörü hazırlanıyor: {TARGET_OUTPUT_DIR}")
    if TARGET_OUTPUT_DIR.exists():
        print(f"Mevcut '{TARGET_OUTPUT_DIR}' klasörü siliniyor.")
        shutil.rmtree(TARGET_OUTPUT_DIR)
    TARGET_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    print(f"'{TARGET_OUTPUT_DIR}' klasörü oluşturuldu/temizlendi.\n")

    # 2. Eğitim çalıştırması klasörünü kopyala
    if not SOURCE_TRAIN_RUN_DIR.is_dir():
        print(f"KRİTİK HATA: Ana eğitim çalıştırma dizini '{SOURCE_TRAIN_RUN_DIR}' bulunamadı!")
        print("Arşivleme işlemi devam edemez. Lütfen TRAIN_EXPERIMENT_NAME ayarını kontrol edin.")
        return
    copy_directory_if_exists(SOURCE_TRAIN_RUN_DIR, TARGET_OUTPUT_DIR)

    # 3. Test metrikleri çalıştırması klasörünü (/content/runs/test/test_on_rebalanced_data) kopyala
    # Bu klasörü, ana çıktı klasörünün içine 'test_evaluation_metrics' gibi bir isimle kopyalayalım
    copy_directory_if_exists(SOURCE_TEST_VAL_METRICS_DIR, TARGET_OUTPUT_DIR, new_folder_name="test_evaluation_metrics")

    # 4. Test tahminleri çalıştırması klasörünü (/content/runs/test/test_on_rebalanced_data_preds) kopyala
    # Bu klasörü, ana çıktı klasörünün içine 'test_predictions_output' gibi bir isimle kopyalayalım
    copy_directory_if_exists(SOURCE_TEST_PREDICTIONS_DIR, TARGET_OUTPUT_DIR, new_folder_name="test_predictions_output")

    # 5. Potansiyel ayrı doğrulama çalıştırması klasörünü (/content/runs/detect/val gibi) kopyala
    if SOURCE_SEPARATE_VAL_DIR: # Sadece bir isim tanımlanmışsa ve klasör varsa kopyala
        copy_directory_if_exists(SOURCE_SEPARATE_VAL_DIR, TARGET_OUTPUT_DIR, new_folder_name="separate_validation_run")

    # 6. (İsteğe Bağlı) En iyi model ağırlığını ana çıktı klasörünün köküne de kopyalayabiliriz
    source_best_pt_path = SOURCE_TRAIN_RUN_DIR / "weights" / "best.pt"
    target_best_pt_main_path = TARGET_OUTPUT_DIR / "best_trained_model.pt"
    if source_best_pt_path.is_file():
        print(f"En iyi model ağırlıkları ayrıca ana çıktı klasörünün köküne kopyalanıyor:")
        print(f"  Kaynak: {source_best_pt_path}")
        print(f"  Hedef:  {target_best_pt_main_path}")
        shutil.copy(source_best_pt_path, target_best_pt_main_path)
        print("Model ağırlıkları (kök kopya) başarıyla kopyalandı.\n")
    else:
        # Bu uyarı zaten eğitim klasörü kopyalanırken de verileceği için tekrar gerekmeyebilir,
        # ancak burada da bulunması zararsızdır.
        print(f"UYARI: En iyi model ağırlık dosyası ({source_best_pt_path}) ana eğitim klasöründe bulunamadı.\n")


    # 7. Hazırlanan hedef çıktı klasörünü sıkıştır
    print(f"Çıktılar sıkıştırılıyor...")
    print(f"  Sıkıştırılacak klasör (base_dir): '{TARGET_OUTPUT_FOLDER_NAME}' (içeriği: {TARGET_OUTPUT_DIR})")
    print(f"  Sıkıştırma kök dizini (root_dir): '{TARGET_ARCHIVE_PARENT_DIR}'")
    print(f"  Oluşturulacak zip dosyası: {FINAL_ZIP_PATH}")

    try:
        if FINAL_ZIP_PATH.exists():
            print(f"Mevcut zip dosyası siliniyor: {FINAL_ZIP_PATH}")
            FINAL_ZIP_PATH.unlink()

        shutil.make_archive(
            base_name=str(TARGET_ARCHIVE_PARENT_DIR / ZIP_FILE_BASENAME),
            format='zip',
            root_dir=str(TARGET_ARCHIVE_PARENT_DIR),
            base_dir=TARGET_OUTPUT_FOLDER_NAME
        )
        print(f"Sıkıştırma tamamlandı. Arşiv dosyası oluşturuldu: {FINAL_ZIP_PATH}\n")
    except Exception as e:
        print(f"Sıkıştırma sırasında bir hata oluştu: {e}\n")
        return

    print("İşlem başarıyla tamamlandı!")
    print(f"Tüm sonuçlar '{FINAL_ZIP_PATH}' dosyasına kaydedildi.")
    print(f"Bu zip dosyasını Colab'ın sol tarafındaki dosya gezgininden indirebilirsiniz.")
    print(f"Zip dosyasını açtığınızda içinde '{TARGET_OUTPUT_FOLDER_NAME}' adlı bir klasör bulacaksınız.")
    print(f"Bu klasörün içinde (eğer kaynakları bulunduysa):\n"
          f"  - '{TRAIN_EXPERIMENT_NAME}/' (ana eğitim çıktıları)\n"
          f"  - 'test_evaluation_metrics/' (test seti değerlendirme metrikleri ve ilgili dosyalar)\n"
          f"  - 'test_predictions_output/' (test seti tahmin görselleri ve .txt dosyaları)\n"
          f"  - 'best_trained_model.pt' (eğitilmiş en iyi modelin bir kopyası)\n"
          + (f"  - 'separate_validation_run/' (ayrı bir doğrulama çalıştırması varsa onun çıktıları)\n" if SOURCE_SEPARATE_VAL_DIR and SOURCE_SEPARATE_VAL_DIR.is_dir() else "")
         )

# Betiği çalıştırmak için:
if __name__ == "__main__":
    collect_and_zip_all_outputs()